In [1]:
%matplotlib notebook
import orbipy as op
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from copy import copy

In [2]:
model = op.crtbp3_model()
precise_model = op.crtbp3_model()
precise_model.integrator.set_params(max_step=np.pi/180)
plotter = op.plotter.from_model(model, length_units='nd', velocity_units='nd/nd')
scaler = plotter.scaler

In [3]:
def interactive_map(data, buffer):
    def onclick(event):
        if event.button == 3:
            ix, iy = event.xdata, event.ydata
            buffer.append([ix, iy])
            ax.plot(ix, iy, 'or')
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(data['x0'], data['z0'], 'ob')
    ax.plot(s0[0],s0[2],'xr')
    ax.axis('equal')
    cid = fig.canvas.mpl_connect('button_press_event', onclick)

In [ ]:
with open('amp.pkl', 'rb') as fp:
    applicability = pickle.load(fp)

In [ ]:
buffer = []
interactive_map(applicability, buffer)

In [ ]:
x0 = buffer[-1][0]
z0 = buffer[-1][1]

In [ ]:
def manifolds(s0, dv, num=200):
    traj_upper = []
    traj_lower = []
    shift = 0
    v_upper = np.linspace(s0[4]+shift, s0[4] + dv, num)
    v_lower = np.linspace(s0[4] - dv, s0[4]-shift, num)[:-1]
    det = op.event_detector(model, [left, right])
    for v in v_upper:
        s1 = s0.copy()
        s1[4] = v
        orb, _ = det.prop(s1, 0.0, 100, last_state='last')
        traj_upper.append(orb)
    for v in v_lower:
        s1 = s0.copy()
        s1[4] = v
        orb, _ = det.prop(s1, 0.0, 100, last_state='last')
        traj_lower.append(orb)
    
    return traj_lower, traj_upper

In [ ]:
trl, tru = manifolds(df.iloc[0].values[1:], 1e-9)

In [ ]:
for orb in tru:
    plt.plot(orb['x'], orb['y'], 'k')
for orb in trl:
    plt.plot(orb['x'], orb['y'], 'b')
plt.plot(df['x'], df['y'], 'r', alpha=0.3)